# Sentiment Classification


### Generate Word Embeddings and retrieve outputs of each layer with Keras based on Classification task

Word embeddings are a type of word representation that allows words with similar meaning to have a similar representation.

It is a distributed representation for text that is perhaps one of the key breakthroughs for the impressive performance of deep learning methods on challenging natural language processing problems.

We willl use the imdb dataset to learn word embeddings as we train our dataset. This dataset contains 25,000 movie reviews from IMDB, labeled with sentiment (positive or negative). 



### Dataset

`from keras.datasets import imdb`

Dataset of 25,000 movies reviews from IMDB, labeled by sentiment (positive/negative). Reviews have been preprocessed, and each review is encoded as a sequence of word indexes (integers). For convenience, the words are indexed by their frequency in the dataset, meaning the for that has index 1 is the most frequent word. Use the first 20 words from each review to speed up training, using a max vocab size of 10,000.

As a convention, "0" does not stand for a specific word, but instead is used to encode any unknown word.


### Aim

1. Import test and train data  
2. Import the labels ( train and test) 
3. Get the word index and then Create key value pair for word and word_id. (12.5 points)
4. Build a Sequential Model using Keras for Sentiment Classification task. (10 points)
5. Report the Accuracy of the model. (5 points)  
6. Retrive the output of each layer in keras for a given single test sample from the trained model you built. (2.5 points)


#### Usage:

In [1]:
from keras.datasets import imdb
from tensorflow.python.keras.layers import Dropout, Dense, Embedding, Flatten, LSTM
from tensorflow.python.keras.models import Sequential

vocab_size = 10000 #vocab size

(X_train, Y_train), (X_test, Y_test) = imdb.load_data(num_words=vocab_size) # vocab_size is no.of words to consider from the dataset, ordering based on frequency.

Using TensorFlow backend.


17473536/17464789 [==============================] - 1s 0us/step


W0105 10:09:25.061767 139736757589888 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/tensorflow_estimator/python/estimator/api/_v1/estimator/__init__.py:12: The name tf.estimator.inputs is deprecated. Please use tf.compat.v1.estimator.inputs instead.



ValueError: ignored

above method is throwing error while loading file so using the below method to import data from dataset


In [0]:
import numpy as np
# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

# call load_data with allow_pickle implicitly set to true
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=10000)

# restore np.load for future normal usage
np.load = np_load_old

In [0]:
x_train.shape

(25000,)

In [0]:
y_train.shape

(25000,)

In [0]:
x_test.shape

(25000,)

In [0]:
from keras.preprocessing.sequence import pad_sequences
vocab_size = 10000 #vocab size
maxlen = 300  #number of word used from each review

In [0]:
#load dataset as a list of ints
#(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)
#make all sequences of the same length
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test =  pad_sequences(x_test, maxlen=maxlen)

In [0]:
x_train.shape

(25000, 300)

In [0]:
y_train.shape

(25000,)

In [0]:
x_test.shape

(25000, 300)

In [0]:
y_test.shape

(25000,)

In [0]:
x_train

array([[   0,    0,    0, ...,   19,  178,   32],
       [   0,    0,    0, ...,   16,  145,   95],
       [   0,    0,    0, ...,    7,  129,  113],
       ...,
       [   0,    0,    0, ...,    4, 3586,    2],
       [   0,    0,    0, ...,   12,    9,   23],
       [   0,    0,    0, ...,  204,  131,    9]], dtype=int32)

## Build Keras Embedding Layer Model
We can think of the Embedding layer as a dicionary that maps a index assigned to a word to a word vector. This layer is very flexible and can be used in a few ways:

* The embedding layer can be used at the start of a larger deep learning model. 
* Also we could load pre-train word embeddings into the embedding layer when we create our model.
* Use the embedding layer to train our own word2vec models.

The keras embedding layer doesn't require us to onehot encode our words, instead we have to give each word a unqiue intger number as an id. For the imdb dataset we've loaded this has already been done, but if this wasn't the case we could use sklearn [LabelEncoder](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html).

In [0]:
#Initialize model
import tensorflow as tf

tf.keras.backend.clear_session()
model = tf.keras.Sequential()

In [0]:
model.add(tf.keras.layers.Embedding(vocab_size + 1, #Vocablury size
                                    50, #Embedding size
                                    input_length=maxlen) #Number of words in each review
          )

W1223 13:59:34.140010 140059347376000 deprecation.py:506] From /usr/local/lib/python2.7/dist-packages/tensorflow_core/python/keras/initializers.py:119: calling __init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1223 13:59:34.161984 140059347376000 deprecation.py:506] From /usr/local/lib/python2.7/dist-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling __init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
model.output

<tf.Tensor 'embedding/embedding_lookup/Identity_1:0' shape=(?, 300, 50) dtype=float32>

In [0]:
model.add(tf.keras.layers.LSTM(256, #RNN State - size of cell state and hidden state
                               dropout=0.2, #Dropout before feeding the data to LSTM layer
                               recurrent_dropout=0.2)) #Dropout applied to the output of LSTM layer

In [0]:
model.output

<tf.Tensor 'lstm/strided_slice_7:0' shape=(?, 256) dtype=float32>

In [0]:
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))

In [0]:
#Compile the model
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

W1223 13:59:37.114236 140059347376000 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow_core/python/ops/nn_impl.py:183: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 300, 50)           500050    
_________________________________________________________________
lstm (LSTM)                  (None, 256)               314368    
_________________________________________________________________
dense (Dense)                (None, 1)                 257       
Total params: 814,675
Trainable params: 814,675
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.fit(x_train,y_train,
          epochs=20,
          batch_size=32,          
          validation_data=(x_test, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/20
25000/25000 [==============================] - 439s 18ms/sample - loss: 0.4979 - acc: 0.7652 - val_loss: 0.4075 - val_acc: 0.8175
Epoch 2/20
25000/25000 [==============================] - 421s 17ms/sample - loss: 0.4644 - acc: 0.7834 - val_loss: 0.9165 - val_acc: 0.5874
Epoch 3/20
25000/25000 [==============================] - 424s 17ms/sample - loss: 0.3668 - acc: 0.8413 - val_loss: 0.3634 - val_acc: 0.8501
Epoch 4/20
25000/25000 [==============================] - 421s 17ms/sample - loss: 0.2739 - acc: 0.8874 - val_loss: 0.3216 - val_acc: 0.8744
Epoch 5/20
25000/25000 [==============================] - 418s 17ms/sample - loss: 0.2083 - acc: 0.9185 - val_loss: 0.3427 - val_acc: 0.8623
Epoch 6/20
25000/25000 [==============================] - 416s 17ms/sample - loss: 0.1728 - acc: 0.9334 - val_loss: 0.3357 - val_acc: 0.8676
Epoch 7/20
25000/25000 [==============================] - 414s 17ms/sample - loss: 0.1446 - acc: 0.9449 

## Retrive the output of each layer in keras for a given single test sample from the trained model you built

In [0]:
from keras import backend as K
                                          # input placeholder
outputs = [layer.output for layer in model.layers]          # all layer outputs


In [0]:
outputs

[<tf.Tensor 'embedding/embedding_lookup/Identity_1:0' shape=(?, 300, 50) dtype=float32>,
 <tf.Tensor 'lstm/strided_slice_7:0' shape=(?, 256) dtype=float32>,
 <tf.Tensor 'dense/Sigmoid:0' shape=(?, 1) dtype=float32>]